In [ ]:
# default_exp base_params
%load_ext autoreload
%autoreload 2
from nbdev.showdoc import show_doc
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# Params

`BaseParams` is the major object to control the whole modeling process. It is supposed to be accessable anywhere. 

In [ ]:

# export

import fcntl
import json
import os
import re
import time
from collections import defaultdict
from distutils.dir_util import copy_tree
from typing import Any, Callable, Dict, List, Tuple, Union

import tensorflow as tf
from loguru import logger
from m3tl.mtl_model.base import BasicMTL
from m3tl.special_tokens import BOS_TOKEN, EOS_TOKEN, TRAIN
from m3tl.utils import (create_path, get_phase, load_transformer_config,
                        load_transformer_tokenizer)


class BaseParams():
    # pylint: disable=attribute-defined-outside-init
    def __init__(self):

        ################ transformers params #################
        self.transformer_model_name = 'bert-base-chinese'
        self.transformer_tokenizer_name = 'bert-base-chinese'
        self.transformer_config_name = 'bert-base-chinese'
        self.transformer_model_loading = 'TFBertModel'
        self.transformer_config_loading = 'BertConfig'
        self.transformer_tokenizer_loading = 'BertTokenizer'
        self.transformer_decoder_model_name = None
        self.transformer_decoder_config_name = None
        self.transformer_decoder_tokenizer_name = None
        # self.transformer_decoder_model_name = "hfl/chinese-xlnet-base"
        # self.transformer_decoder_config_name = "hfl/chinese-xlnet-base"
        # self.transformer_decoder_tokenizer_name = "hfl/chinese-xlnet-base"
        self.transformer_decoder_model_loading = 'TFAutoModel'
        self.transformer_decoder_config_loading = 'AutoConfig'
        self.transformer_decoder_tokenizer_loading = 'AutoTokenizer'
        ################ transformers params end ###############

        ################ multi-model configuration #############
        # multimodal params
        self.modal_type_id = {
            'text': 0,
            'image': 1,
            'others': 2
        }
        self.enable_modal_type = False
        ############### multi-modal configuration end ##########
        # bert config
        self.init_checkpoint = ''

        ############### registration information #############
        # problem type
        self.problem_type_list = []
        self.predefined_problem_type = []
        self.get_or_make_label_encoder_fn_dict: Dict[str, Callable] = {}
        self.label_handling_fn: Dict[str, Callable] = {}
        self.problem_type_desc : Dict[str, str] = {}
        self.top_layer = {}
        self.num_classes = {}
        self.problem_info = defaultdict(dict)
        self.problem_type = {}

        # MTL models
        self.available_mtl_model = {}
        self.mtl_model = {}
        self.register_mtl_model('basic', BasicMTL, include_top=False)
        self.assign_mtl_model('basic')

        # problem sampling strategy
        self.available_problem_sampling_strategy = {}
        self.problem_sampling_strategy = {}

        # loss combination strategy
        self.available_loss_combination_strategy = {}
        self.loss_combination_strategy = {}

        # gradient surgery
        self.available_gradient_surgery = {}
        self.gradient_surgery = {}

        # embedding layer
        self.available_embedding_layer = {}
        self.embedding_layer = {}
        ############### registration information end ###########

        ############### logging control #############
        self.detail_log = True
        self.print_data = True
        ############### logging control end #############

        ############### model hparam #############
        self.init_lr = 2e-5
        self.dropout = 0.1
        self.dropout_keep_prob = 1 - self.dropout
        self.max_seq_len = 256
        self.use_one_hot_embeddings = True
        self.label_smoothing = 0.0
        self.crf = False
        self.bert_num_hidden_layer = 12
        self.hidden_dense = False
        # threshold to calculate metrics for multi_cls
        self.multi_cls_threshold = 0.5
        self.multi_cls_positive_weight = 1.0
        self.custom_pooled_hidden_size = 0
        self.share_embedding = True
        self.output_problem_pred = True
        self.output_body_seq_hidden = False
        self.output_body_pooled_hidden = False
        self.output_mtl_model_hidden = False
        # DuplicateAugMultimodalEmbedding parameters
        self.duplicate_data_aug_problems: Union[List[str], str] = None
        # Contrastive Learning parameters
        self.contrastive_learning_model_name: str = None
        self.simcse_pooler = 'pooled'
        ############### model hparam end #############

        ############### training config #############
        self.num_cpus = 1
        self.preprocess_buffer = 100000
        self.example_per_file = 100000
        self.decode_vocab_file = None
        self.batch_size = 32
        self.train_epoch = 15
        self.freeze_step = 0
        self.prefetch = 5000
        self.dynamic_padding = True
        self.bucket_batch_sizes = [32, 32, 32, 16]
        self.bucket_boundaries = [30, 64, 128]
        self.shuffle_buffer = 200000
        self.train_steps = 0
        self.num_warmup_steps = 0
        self.use_horovod = False
        ############### training config end #############

        ############### runtime assigned_details #############
        self.run_problem_list = []
        self.assigned_details = ()
        self.problem_assigned = False
        self.predicting = False
        self.problem_list = []
        self.problem_chunk = []
        self.problem_str = None
        ############### runtime assigned details end #############

        ############### assign details #############
        self.model_dir = None
        self.ckpt_dir = ''
        self.params_path = ''
        self.bert_config = None
        self.bert_config_dict = None
        self.bert_decoder_config = None
        self.bert_decoder_config_dict = None
        self.init_weight_from_huggingface = True
        self.vocab_size = None
        self.decoder_vocab_size = None
        self.bos_id = None
        self.eos_id = None

        ############### assign details end #############

        self.tmp_file_dir = 'tmp'
        # get generator function for each problem
        self.read_data_fn = {}
        self.problem_assigned = False

        # pyspark related
        self.pyspark_output_path = None

        ############### legacy parameters #############
        # seq2seq
        self.decoder_num_hidden_layers = 3
        self.beam_size = 10
        self.init_decoder_from_encoder = False
        self.beam_search_alpha = 0.6
        self.decode_max_seq_len = 90

        # pretrain hparm
        self.dupe_factor = 10
        self.short_seq_prob = 0.1
        self.masked_lm_prob = 0.15
        self.max_predictions_per_seq = 20
        self.mask_lm_hidden_size = 768
        self.mask_lm_hidden_act = 'gelu'
        self.mask_lm_initializer_range = 0.02
        ############### legacy parameters end #############

    def get(self, key, default=None):
        return getattr(self, key, default)

    def to_json(self):
        """Save the params as json files. Please note that processing_fn is not saved.
        """
        dump_dict = {}
        for att_name, att in vars(self).items():
            try:
                json.dumps(att)
                dump_dict[att_name] = att
            except TypeError:
                pass

        with open(self.params_path, 'w', encoding='utf8') as f:
            json.dump(dump_dict, f, indent=4)

    def from_json(self, json_path: str = None):
        """Load json file as params.

        json_path could not be None if the problem is not assigned to params

        Args:
            json_path (str, optional): Path to json file. Defaults to None.

        Raises:
            AttributeError
        """
        try:
            params_path = json_path if json_path is not None else self.params_path
        except AttributeError:
            AttributeError(
                'Either json_path should not be None or problem is assigned.')

        if self.problem_assigned:
            assign_details = self.assigned_details
        else:
            assign_details = None

        with open(params_path, 'r', encoding='utf8') as f:
            dump_dict = json.load(f)
        for att in dump_dict:
            setattr(self, att, dump_dict[att])
        self.bert_config = load_transformer_config(
            self.bert_config_dict, self.transformer_config_loading)
        if self.bert_decoder_config_dict:
            self.bert_decoder_config = load_transformer_config(
                self.bert_decoder_config_dict, self.transformer_decoder_config_loading
            )
        if assign_details:
            self.assign_problem(*assign_details)

    def parse_problem_string(self, flag_string: str) -> Tuple[List[str], List[List[str]]]:

        self.problem_str = flag_string
        problem_list = sorted(re.split(r'[&|]', flag_string))

        for problem in problem_list:
            if problem not in self.problem_type:
                raise KeyError("problem {} is not registered.".format(problem))

        # Parse problem string
        self.run_problem_list = []
        problem_chunk = []
        for flag_chunk in flag_string.split('|'):

            if '&' not in flag_chunk:
                problem_type = {}
                problem_type[flag_chunk] = self.problem_type[flag_chunk]
                self.run_problem_list.append(problem_type)
                problem_chunk.append([flag_chunk])
            else:
                problem_type = {}
                problem_chunk.append([])
                for problem in flag_chunk.split('&'):
                    problem_type[problem] = self.problem_type[problem]
                    problem_chunk[-1].append(problem)
                self.run_problem_list.append(problem_type)

        return problem_list, problem_chunk

    @staticmethod
    def _copy_or_wait(from_path, to_path, do_copy=True, max_wait=5, replace=False):
        if os.path.exists(to_path) and replace is False:
            return
        if do_copy:
            copy_tree(from_path, to_path)
            return
        else:
            for _ in range(max_wait):
                if not os.path.exists(to_path):
                    time.sleep(10)
                else:
                    return

    def prepare_dir(self, base_dir: str, dir_name: str, problem_list: List[str], copy_dir=True):
        """prepare model checkpoint dir. this function will copy or save transformers' configs
        and tokenizers to params.ckpt_dir

        Args:
            base_dir (str): base_dir of params.ckpt_dir. same as os.path.dirname(params.ckpt_dir). bad naming
            dir_name (str): dir_name, same as os.path.basename(params.ckpt_dir). bad naming
            problem_list (List[str]): [description]
        """
        base = base_dir if base_dir is not None else 'models'

        dir_name = dir_name if dir_name is not None else '_'.join(
            problem_list)+'_ckpt'
        self.ckpt_dir = os.path.join(base, dir_name)

        # we need to make sure all configs, tokenizers are in ckpt_dir
        # configs
        from_config_path = os.path.join(self.init_checkpoint,
                                        'bert_config')
        from_decoder_config_path = os.path.join(self.init_checkpoint,
                                                'bert_decoder_config')
        to_config_path = os.path.join(self.ckpt_dir, 'bert_config')
        to_decoder_config_path = os.path.join(
            self.ckpt_dir, 'bert_decoder_config')

        # tokenizers
        from_tokenizer_path = os.path.join(self.init_checkpoint, 'tokenizer')
        to_tokenizer_path = os.path.join(self.ckpt_dir, 'tokenizer')

        from_decoder_tokenizer_path = os.path.join(
            self.init_checkpoint, 'decoder_tokenizer')
        to_decoder_tokenizer_path = os.path.join(
            self.ckpt_dir, 'decoder_tokenizer')

        self.params_path = os.path.join(self.ckpt_dir, 'params.json')

        if not self.predicting:
            create_path(self.ckpt_dir)

            # two ways to init model
            # 1. init from TF checkpoint dir created by m3tl.
            # 2. init from huggingface checkpoint.

            # bert config exists, init from existing config
            if os.path.exists(from_config_path):
                # copy config
                self._copy_or_wait(
                    from_config_path, to_config_path, do_copy=copy_dir)
                self.bert_config = load_transformer_config(
                    to_config_path, self.transformer_config_loading)

                # copy tokenizer
                self._copy_or_wait(from_tokenizer_path,
                                   to_tokenizer_path, do_copy=copy_dir)

                # copy decoder config
                if os.path.exists(from_decoder_config_path):
                    self._copy_or_wait(
                        from_decoder_config_path, to_decoder_config_path, do_copy=copy_dir)
                    self.bert_decoder_config = load_transformer_config(
                        from_decoder_config_path, self.transformer_decoder_config_loading
                    )
                    self.bert_decoder_config_dict = self.bert_decoder_config.to_dict()
                # copy decoder tokenizer
                if os.path.exists(from_decoder_tokenizer_path):
                    self._copy_or_wait(
                        from_decoder_tokenizer_path, to_decoder_tokenizer_path, do_copy=copy_dir)

                self.init_weight_from_huggingface = False
            else:
                # load config from huggingface
                logger.warning(
                    '{} not exists. will load model from huggingface checkpoint.', from_config_path)
                # get or download config
                self.init_weight_from_huggingface = True
                self.bert_config = load_transformer_config(
                    self.transformer_config_name, self.transformer_config_loading)
                self.bert_config.save_pretrained(to_config_path)

                # save tokenizer
                tokenizer = load_transformer_tokenizer(
                    self.transformer_tokenizer_name, self.transformer_tokenizer_loading)
                tokenizer.save_pretrained(to_tokenizer_path)
                # save_pretrained method of tokenizer saves the config as tokenizer_config.json, which will cause
                # OSError if use tokenizer.from_pretrained directly. we need to manually rename the json file
                try:
                    os.rename(os.path.join(to_tokenizer_path, 'tokenizer_config.json'), os.path.join(
                        to_tokenizer_path, 'config.json'))
                except:
                    pass

                # if decoder is specified
                if self.transformer_decoder_model_name:
                    self.bert_decoder_config = load_transformer_config(
                        self.transformer_decoder_config_name, self.transformer_decoder_config_loading
                    )
                    self.bert_decoder_config_dict = self.bert_decoder_config.to_dict()
                    self.bert_decoder_config.save_pretrained(
                        to_decoder_config_path)
                    decoder_tokenizer = load_transformer_tokenizer(
                        self.transformer_decoder_tokenizer_name, self.transformer_decoder_tokenizer_loading)
                    decoder_tokenizer.save_pretrained(
                        to_decoder_tokenizer_path)
                    try:
                        os.rename(os.path.join(to_decoder_tokenizer_path, 'tokenizer_config.json'), os.path.join(
                            to_decoder_tokenizer_path, 'config.json'))
                    except:
                        pass
        else:
            self.bert_config = load_transformer_config(to_config_path)
            if os.path.exists(to_decoder_config_path):
                self.bert_decoder_config = load_transformer_config(
                    to_decoder_config_path)
            self.init_weight_from_huggingface = False

        self.transformer_config_name = to_config_path
        # set value if and only if decoder is assigned
        self.transformer_decoder_config_name = to_decoder_config_path if self.transformer_decoder_config_name is not None else None
        self.transformer_tokenizer_name = to_tokenizer_path
        # set value if and only if decoder is assigned
        self.transformer_decoder_tokenizer_name = to_decoder_tokenizer_path if self.transformer_decoder_tokenizer_name is not None else None

        self.bert_config_dict = self.bert_config.to_dict()

        tokenizer = load_transformer_tokenizer(
            self.transformer_tokenizer_name, self.transformer_tokenizer_loading)
        self.vocab_size = tokenizer.vocab_size
        if self.transformer_decoder_tokenizer_name:
            decoder_tokenizer = load_transformer_tokenizer(
                self.transformer_decoder_tokenizer_name,
                self.transformer_decoder_tokenizer_loading
            )

            # if set bos and eos
            if decoder_tokenizer.bos_token is None:
                decoder_tokenizer.add_special_tokens({'bos_token': BOS_TOKEN})

            if decoder_tokenizer.eos_token is None:
                decoder_tokenizer.add_special_tokens({'eos_token': EOS_TOKEN})

            # overwrite tokenizer
            decoder_tokenizer.save_pretrained(to_decoder_tokenizer_path)

            self.decoder_vocab_size = decoder_tokenizer.vocab_size
            self.bos_id = decoder_tokenizer.bos_token_id
            self.eos_id = decoder_tokenizer.eos_token_id

    def get_problem_type(self, problem: str) -> str:
        return self.problem_type[problem]

    def update_train_steps(self, train_steps_per_epoch: int, epoch: int = None, warmup_ratio=0.1) -> None:
        """If the batch_size is dynamic, we have to loop through the tf.data.Dataset
        to get the accurate number of training steps. In this case, we need a function to
        update the train_steps which will be used to calculate learning rate schedule.

        WARNING: updating should be called before the model is compiled!

        Args:
            train_steps (int): new number of train_steps
        """
        if epoch:
            train_steps = train_steps_per_epoch * epoch
        else:
            train_steps = train_steps_per_epoch * self.train_epoch

        logger.critical('Updating train_steps to {}'.format(train_steps))

        self.train_steps = train_steps
        self.train_steps_per_epoch = train_steps_per_epoch
        self.num_warmup_steps = int(self.train_steps * warmup_ratio)

    def get_problem_chunk(self, problem: str = None, as_str=True) -> Union[List[str], List[List[str]]]:

        # get chunk of the problem
        if problem is not None:
            all_problem_chunk_as_str = self.get_problem_chunk(as_str=True)
            # if problem itself is actually a chunk
            if problem in all_problem_chunk_as_str:
                return [problem]
            problem_chunk_to_search = [
                c for c in self.problem_chunk if problem in c]
        else:
            problem_chunk_to_search = self.problem_chunk

        if len(problem_chunk_to_search) == 0:
            raise ValueError('problem {} not in problem chunk {}'.format(
                problem, self.problem_chunk))

        if as_str:
            res_list = []
            for problem_list in problem_chunk_to_search:
                res_list.append('_'.join(sorted(problem_list)))
            return res_list
        else:
            return problem_chunk_to_search

    def register_problem_type(self,
                              problem_type: str,
                              top_layer: tf.keras.Model = None,
                              label_handling_fn: Callable = None,
                              get_or_make_label_encoder_fn: Callable = None,
                              inherit_from: str = None,
                              description: str = None):
        self.problem_type_list.append(problem_type)

        if inherit_from is not None:
            assert inherit_from in self.problem_type_list, 'try to inherit from {0}, but {0} is not registered.'.format(
                inherit_from)

        if get_or_make_label_encoder_fn is None and inherit_from is not None:
            self.get_or_make_label_encoder_fn_dict[problem_type] = self.get_or_make_label_encoder_fn_dict[inherit_from]
        else:
            if not isinstance(get_or_make_label_encoder_fn, Callable):
                raise TypeError(
                    'get_or_make_label_encoder_fn of problem type {} should be callable'.format(problem_type))
            self.get_or_make_label_encoder_fn_dict[problem_type] = get_or_make_label_encoder_fn

        if top_layer is None and inherit_from is not None:
            self.top_layer[problem_type] = self.top_layer[inherit_from]
        else:
            # if not isinstance(top_layer, tf.keras.Model) and not isinstance(top_layer, tf.keras.layers.Layer):
            #     raise TypeError(
            #         'top_layer of problem {} should be `tf.keras.Model` or `tf.keras.layers.Layer`'.format(problem_type))
            self.top_layer[problem_type] = top_layer

        if label_handling_fn is None and inherit_from is not None:
            self.label_handling_fn[problem_type] = self.label_handling_fn[inherit_from]
        else:
            if not isinstance(label_handling_fn, Callable):
                raise TypeError(
                    'label_handling_fn of problem type {} should be callable'.format(problem_type))
            self.label_handling_fn[problem_type] = label_handling_fn
        
        self.problem_type_desc[problem_type] = description

    def list_available_problem_types(self) -> List[str]:
        return self.problem_type_list

    def register_problem(self, problem_name: str, problem_type='cls', processing_fn: Callable = None):

        if problem_type not in self.problem_type_list:
            raise ValueError('Provided problem type not valid, expect {0}, got {1}'.format(
                self.problem_type_list,
                problem_type))

        self.problem_type[problem_name] = problem_type
        self.read_data_fn[problem_name] = processing_fn

    def register_multiple_problems(self, problem_type_dict: Dict[str, str], processing_fn_dict: Dict[str, Callable] = None):
        # add new problem to params if problem_type_dict and processing_fn_dict provided
        for new_problem, problem_type in problem_type_dict.items():
            logger.info('Adding new problem {0}, problem type: {1}'.format(
                new_problem, problem_type_dict[new_problem]))
            if processing_fn_dict:
                new_problem_processing_fn = processing_fn_dict[new_problem]
            else:
                new_problem_processing_fn = None
            self.register_problem(
                problem_name=new_problem, problem_type=problem_type, processing_fn=new_problem_processing_fn)

    @staticmethod
    def _append_info_to_file(path: str, problem: str, info_name: str, info: Any):
        if ':' in info_name:
            raise ValueError('info_name cannot contain :')
        if not os.path.exists(path):
            os.makedirs(os.path.dirname(path), exist_ok=True)
        with open(path, 'a', encoding='utf8') as f:
            fcntl.flock(f, fcntl.LOCK_EX)
            info_str = json.dumps(info)
            f.write('{}:{}:{}\n'.format(problem, info_name, info_str))
            fcntl.flock(f, fcntl.LOCK_UN)

    @staticmethod
    def _read_info_file(path: str) -> dict:
        res = defaultdict(dict)
        if not os.path.exists(path):
            return res
        with open(path, 'r', encoding='utf8') as f:
            for line in f:
                problem, key, val = line.split(':', 2)
                val = json.loads(val)
                res[problem][key] = val
        return res

    def merge_problem_info_file(self, path: str):
        """merge a problem info file"""
        income_problem_info = BaseParams._read_info_file(path)
        for problem in income_problem_info:
            for info_name, info in income_problem_info[problem].items():
                self.set_problem_info(
                    problem=problem, info_name=info_name, info=info)

    def set_problem_info(self, problem: str, info_name: str, info: Any):
        """api to save problem-specific info and can be access anywhere"""
        if not problem or get_phase() != TRAIN:
            return

        # if json exists, load json
        problem_info_json_path = self.get_problem_info_path(problem=problem)

        try:
            old_info = self.get_problem_info(
                problem=problem, info_name=info_name)
            if old_info != info:
                raise KeyError
        except KeyError:
            self._append_info_to_file(
                path=problem_info_json_path, problem=problem, info_name=info_name, info=info)
            self.problem_info.update(
                self._read_info_file(problem_info_json_path))

    def get_problem_info_path(self, problem) -> str:
        problem_chunk = self.get_problem_chunk(problem=problem, as_str=True)[0]
        return os.path.join(self.tmp_file_dir, problem_chunk, 'problem_info.txt')

    def get_problem_info(self, problem: str = None, info_name: str = None) -> Any:
        if problem is None:
            # load all jsons
            for problem_chunk in self.get_problem_chunk():
                self.get_problem_info(problem=problem_chunk)
            for problem in self.problem_list:
                self.get_problem_info(problem=problem)
            return self.problem_info

        problem_info_json_path = self.get_problem_info_path(problem=problem)
        if os.path.exists(problem_info_json_path):
            self.problem_info.update(
                self._read_info_file(problem_info_json_path))

        if problem not in self.problem_info:
            err_str = ("cannot get {0}'s problem info, seems you have not created tfrecord for problem. \n"
                       "please make sure either: \n"
                       "    1. path {1} exists. this is usually created when creating tfrecord for problem. \n"
                       "    2. {0} exists in 'problem_info' key of {2}. ".format(problem, problem_info_json_path, self.params_path))
            raise KeyError(err_str)

        if info_name is None:
            return self.problem_info[problem]

        if info_name not in self.problem_info[problem]:
            raise KeyError(
                "problem {} dose not have info {}".format(problem, info_name))

        return self.problem_info[problem][info_name]

    def assign_problem(self,
                       flag_string: str,
                       model_dir: str = None,
                       base_dir: str = None,
                       dir_name: str = None,
                       predicting=False):

        if model_dir:
            base_dir, dir_name = os.path.split(model_dir)
        elif base_dir or dir_name:
            logger.warning(
                "base_dir and dir_name arguments will be deprecated in the future. Please use model_dir instead.")

        self.assigned_details = (
            flag_string, model_dir, base_dir, dir_name, predicting)
        self.problem_assigned = True
        self.predicting = predicting

        self.problem_list, self.problem_chunk = self.parse_problem_string(
            flag_string)

        # create dir and get vocab, config
        if self.use_horovod:
            import horovod.tensorflow.keras as hvd
            if hvd.rank() == 0:
                self.prepare_dir(base_dir, dir_name, self.problem_list)
            else:
                self.prepare_dir(base_dir, dir_name,
                                 self.problem_list, copy_dir=False)

            self.lr = self.init_lr * hvd.size()
        else:
            self.prepare_dir(base_dir, dir_name, self.problem_list)
            self.lr = self.init_lr * \
                len(tf.config.experimental.list_physical_devices('GPU'))

    def list_available_problems(self) -> Dict[str, str]:
        return self.problem_type

    def assign_data_sampling_strategy(self,
                                      sampling_strategy_name='data_balanced',
                                      sampling_strategy_fn: Callable = None):
        self.problem_sampling_strategy['name'] = sampling_strategy_name
        self.problem_sampling_strategy['fn'] = sampling_strategy_fn

    def calculate_data_sampling_prob(self) -> Dict[str, float]:

        problem_chunk_data_num = defaultdict(float)
        sampling_strategy_name = self.problem_sampling_strategy['name']
        if sampling_strategy_name == 'data_balanced':
            problem_chunk = self.get_problem_chunk(as_str=True)
            for problem_list in problem_chunk:
                problem_chunk_data_num[problem_list] += self.get_problem_info(
                    problem=problem_list, info_name='data_num')
        elif sampling_strategy_name == 'problem_balanced':
            problem_chunk = self.get_problem_chunk(as_str=True)
            for str_per_chunk in problem_chunk:
                problem_chunk_data_num[str_per_chunk] = 1
        else:
            raise ValueError(
                'sampling strategy {} is not implemented by default. '
                'please provide sampling_strategy_fn.'.format(sampling_strategy_name))

        # devided by sum to get sampling prob
        sum_across_problems = sum(
            [v for _, v in problem_chunk_data_num.items()])
        self.problem_sampling_weight_dict = {
            k: v / sum_across_problems for k, v in problem_chunk_data_num.items()}
        return self.problem_sampling_weight_dict

    def list_available_mtl_setup(self) -> Dict[str, list]:
        ret_dict = {}
        ret_dict['available_mtl_model'] = list(self.available_mtl_model.keys())
        ret_dict['available_problem_sampling_strategy'] = list(
            self.available_problem_sampling_strategy.keys())
        ret_dict['available_loss_combination_strategy'] = list(
            self.available_loss_combination_strategy.keys())
        ret_dict['available_gradient_surgery'] = list(
            self.available_gradient_surgery)
        return ret_dict

    def register_mtl_model(
            self,
            model_name: str,
            model: tf.keras.Model,
            include_top: bool,
            extra_info: dict = None):
        self.available_mtl_model[model_name] = {}
        self.available_mtl_model[model_name]['model'] = model
        self.available_mtl_model[model_name]['include_top'] = include_top
        self.available_mtl_model[model_name]['extra_info'] = extra_info
        self.available_mtl_model[model_name]['model_name'] = model_name

    def assign_mtl_model(self, model_name: str):
        if model_name not in self.available_mtl_model:
            raise KeyError('MTL model {} is not registered. Available MTL model: {}'.format(
                model_name, self.available_mtl_model.keys()))
        self.mtl_model = self.available_mtl_model[model_name]

    def register_loss_combination_strategy(
            self,
            name: str,
            model: tf.keras.Model,
            callbacks: List[tf.keras.callbacks.Callback] = None,
            extra_info: dict = None):
        self.available_loss_combination_strategy[name] = {}
        self.available_loss_combination_strategy[name]['model'] = model
        self.available_loss_combination_strategy[name]['name'] = name
        self.available_loss_combination_strategy[name]['callbacks'] = callbacks
        self.available_loss_combination_strategy[name]['extra_info'] = extra_info

    def assign_loss_combination_strategy(
            self,
            name: str):
        if name not in self.available_loss_combination_strategy:
            raise KeyError('Loss combination strategy {} is not registered. Available strategy: {}'.format(
                name, self.available_loss_combination_strategy.keys()))
        self.loss_combination_strategy = self.available_loss_combination_strategy[name]

    def register_embedding_layer(self, name: str, embedding_layer: tf.keras.Model, **kwargs):
        self.available_embedding_layer[name] = {}
        self.available_embedding_layer[name]['model'] = embedding_layer
        self.available_embedding_layer[name]['name'] = name

        for key, arg in kwargs.items():
            self.available_embedding_layer[name][key] = arg

    def assign_embedding_layer(self, name: str):
        if name not in self.available_embedding_layer:
            raise KeyError('Embedding layer {} is not registered. Available embedding_layer: {}'.format(
                name, self.available_embedding_layer.keys()))
        self.embedding_layer = self.available_embedding_layer[name]

    def get_current_setup(self) -> Dict[str, str]:
        setup_dict = {}
        if self.problem_assigned:
            setup_dict['assigned_problems'] = self.assigned_details[0]

        setup_dict['mtl_model'] = self.mtl_model['model_name']
        setup_dict['problem_sampling_strategy'] = self.problem_sampling_strategy.get(
            'name', 'data_balanced')
        setup_dict['loss_combination_strategy'] = self.loss_combination_strategy.get(
            'name', 'sum')
        setup_dict['gradient_surgery'] = self.gradient_surgery.get(
            'name', 'average')

        return setup_dict

    def gather_mtl_callbacks(self):
        callbacks = []
        loss_callback = self.loss_combination_strategy['callbacks']
        if loss_callback:
            callbacks.append(loss_callback)
        return callbacks


class CRFParams(BaseParams):
    def __init__(self):
        super(CRFParams, self).__init__()
        self.crf = True


class StaticBatchParams(BaseParams):
    def __init__(self):
        super(StaticBatchParams, self).__init__()
        self.dynamic_padding = False


class DynamicBatchSizeParams(BaseParams):
    def __init__(self):
        super(DynamicBatchSizeParams, self).__init__()
        self.bucket_batch_sizes = [128, 64, 32, 16]


In [ ]:
# hide
from m3tl.test_base import TestBase
tb = TestBase()
params = tb.params
tmp_model_dir = tb.tmpckptdir

2021-06-15 17:01:36.062 | INFO     | m3tl.base_params:register_multiple_problems:538 - Adding new problem weibo_fake_ner, problem type: seq_tag
2021-06-15 17:01:36.062 | INFO     | m3tl.base_params:register_multiple_problems:538 - Adding new problem weibo_fake_multi_cls, problem type: multi_cls
2021-06-15 17:01:36.063 | INFO     | m3tl.base_params:register_multiple_problems:538 - Adding new problem weibo_fake_cls, problem type: cls
2021-06-15 17:01:36.063 | INFO     | m3tl.base_params:register_multiple_problems:538 - Adding new problem weibo_masklm, problem type: masklm
2021-06-15 17:01:36.064 | INFO     | m3tl.base_params:register_multiple_problems:538 - Adding new problem weibo_fake_regression, problem type: regression
2021-06-15 17:01:36.064 | INFO     | m3tl.base_params:register_multiple_problems:538 - Adding new problem weibo_fake_vector_fit, problem type: vector_fit
2021-06-15 17:01:36.065 | INFO     | m3tl.base_params:register_multiple_problems:538 - Adding new problem weibo_pre

In [ ]:
# hide
# test inherit problem type
params.register_problem_type(problem_type='clone_cls', inherit_from='cls')
assert params.top_layer['cls'] == params.top_layer['clone_cls']

# test correct error message of get_problem_info
try:
    params.get_problem_info(problem='abcd')
except ValueError:
    pass